<h1><center> Retrieval based Voice Conversion WebUI Training Notebook

vist original colab here -> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb)

In [ ]:
# @title mount drive (optional)

# from google.colab import drive

# drive.mount("/content/drive")

In [23]:
# !pip install ipywidgets antlr4-python3-runtime==4.9.3


In [24]:
# !pip install git+https://github.com/antlr4-go/antlr.git
# !pip install ipywidgets antlr4-python3-runtime==4.9.3 scipy

In [ ]:
# @title install (Windows compatible)
from IPython.display import clear_output
from ipywidgets import Button
import subprocess, shlex, os, sys

var = "WebUI"
test = "Voice"
c_word = "Conversion"
r_word = "Retrieval"

repo_url = f"https://github.com/RVC-Project/{r_word}-based-{test}-{c_word}-{var}"
repo_dir = os.path.join(os.getcwd(), "RVC_")

if not os.path.exists(repo_dir):
    subprocess.run(["git", "clone", repo_url, repo_dir])

os.chdir(repo_dir)

pretrains = ["f0D32k.pth", "f0G32k.pth"]
new_pretrains = ["f0Ov2Super32kD.pth", "f0Ov2Super32kG.pth"]

pretrain_dir = os.path.join(repo_dir, "assets", "pretrained_v2")
os.makedirs(pretrain_dir, exist_ok=True)

def download_file(url, output_path):
    if sys.platform == "win32":
        # Use curl for Windows
        cmd = f'curl -L "{url}" -o "{output_path}"'
    else:
        # Use aria2c for Linux
        cmd = f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{os.path.dirname(output_path)}" -o "{os.path.basename(output_path)}"'
    try:
        subprocess.run(cmd, shell=True, check=True)
    except Exception as e:
        print(e)

for file in pretrains:
    dest = os.path.join(pretrain_dir, file)
    if not os.path.exists(dest):
        url = f"https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/{file}"
        download_file(url, dest)

for file in new_pretrains:
    dest = os.path.join(pretrain_dir, file)
    if not os.path.exists(dest):
        url = f"https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/{file}"
        download_file(url, dest)

# Install dependencies
if sys.platform == "win32":
    subprocess.run([sys.executable, "-m", "pip", "install", "pip==23.3.1"])
    subprocess.run(["git", "pull"])
    subprocess.run(["curl", "-L", "https://huggingface.co/Rejekts/project/resolve/main/download_files.py", "-o", os.path.join(repo_dir, "download_files.py")])
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    subprocess.run([sys.executable, "-m", "pip", "install", "yt_dlp"])
    subprocess.run([sys.executable, "-m", "pip", "install", "pytube", "pydub", "gradio==3.42.0"])
    subprocess.run([sys.executable, "download_files.py"])
    subprocess.run([sys.executable, "tools/download_models.py"])
else:
    !pip install pip==23.3.1
    !git pull
    !wget -nc https://huggingface.co/Rejekts/project/resolve/main/download_files.py -O ./download_files.py
    !pip install -r requirements.txt
    !pip install yt_dlp
    !pip install pytube pydub gradio==3.42.0
    !python ./download_files.py
    !python ./tools/download_models.py

clear_output()
Button(description="\u2714 Success", button_style="success")

Button(button_style='success', description='✔ Success', style=ButtonStyle())

# Training No UI

In [ ]:
# Convert to wav
from pydub import AudioSegment
import os
from pydub.utils import which

# Configurar FFmpeg
AudioSegment.converter = which("ffmpeg")

input_folder = r'C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_dataset'
output_folder = r'C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_dataset_wav'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith('.mp3'):
        mp3_path = f"{input_folder}/{filename}"
        wav_filename = os.path.splitext(filename)[0] + '.wav'
        wav_path = f"{output_folder}/{wav_filename}"
        try:
            audio = AudioSegment.from_mp3(mp3_path)
            audio.export(wav_path, format='wav')
            print(f'Convertido: {filename} -> {wav_filename}')
        except FileNotFoundError:
            print(f'Error: FFmpeg no está instalado o no está en el PATH del sistema.')
        except Exception as e:
            print(f'Error con {filename}: {e}')

Convertido: Audio 1.mp3 -> Audio 1.wav
Convertido: Audio 10.mp3 -> Audio 10.wav
Convertido: Audio 11.mp3 -> Audio 11.wav
Convertido: Audio 12.mp3 -> Audio 12.wav
Convertido: Audio 13.mp3 -> Audio 13.wav
Convertido: Audio 11.mp3 -> Audio 11.wav
Convertido: Audio 12.mp3 -> Audio 12.wav
Convertido: Audio 13.mp3 -> Audio 13.wav
Convertido: Audio 14.mp3 -> Audio 14.wav
Convertido: Audio 15.mp3 -> Audio 15.wav
Convertido: Audio 16.mp3 -> Audio 16.wav
Convertido: Audio 14.mp3 -> Audio 14.wav
Convertido: Audio 15.mp3 -> Audio 15.wav
Convertido: Audio 16.mp3 -> Audio 16.wav
Convertido: Audio 17.mp3 -> Audio 17.wav
Convertido: Audio 18.mp3 -> Audio 18.wav
Convertido: Audio 19.mp3 -> Audio 19.wav
Convertido: Audio 17.mp3 -> Audio 17.wav
Convertido: Audio 18.mp3 -> Audio 18.wav
Convertido: Audio 19.mp3 -> Audio 19.wav
Convertido: Audio 2.mp3 -> Audio 2.wav
Convertido: Audio 20.mp3 -> Audio 20.wav
Convertido: Audio 2.mp3 -> Audio 2.wav
Convertido: Audio 20.mp3 -> Audio 20.wav
Convertido: Audio 3.mp

In [6]:
# =============================================================================
# 🔧 PREPROCESAMIENTO DEL DATASET / DATASET PREPROCESSING
# =============================================================================
# Esta celda prepara el dataset de audio para el entrenamiento RVC
# Incluye configuración de paths, normalización de audio y extracción inicial

import os
import sys

# =============================================================================
# 📁 CONFIGURACIÓN DE RUTAS Y PATHS / PATH CONFIGURATION
# =============================================================================

# Configurar rutas principales del proyecto
curr_path = 'C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_proyect'
dataset_folder = 'C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_dataset_wav'
model_name = 'My-Voice'

# Configurar path de RVC para importaciones
rvc_path = os.path.join(curr_path, 'RVC')
print(f"📂 RVC Path: {rvc_path}")

# Agregar RVC al Python path si no está ya
if rvc_path not in sys.path:
    sys.path.insert(0, rvc_path)
    print("✅ RVC path agregado al sistema")

# Cambiar al directorio RVC para imports relativos
original_cwd = os.getcwd()
print(f"📂 Directorio original: {original_cwd}")

try:
    os.chdir(rvc_path)
    print(f"📂 Cambiado a directorio RVC: {os.getcwd()}")
    
    # =============================================================================
    # 📦 IMPORTACIÓN DE MÓDULOS RVC / RVC MODULES IMPORT
    # =============================================================================
    
    # Ahora importar la función de preprocesamiento
    print("📥 Importando función de preprocesamiento...")
    from infer.modules.train.preprocess import preprocess_trainset
    print("✅ Función preprocess_trainset importada exitosamente")
    
    # =============================================================================
    # ⚙️ CONFIGURACIÓN DE PARÁMETROS / PARAMETER CONFIGURATION
    # =============================================================================
    
    # Parámetros del preprocesamiento
    sample_rate = 32000      # Frecuencia de muestreo (32kHz recomendado)
    num_threads = 2          # Número de hilos para procesamiento paralelo
    log_path = f'{curr_path}/RVC/logs/{model_name}'  # Ruta de logs del experimento
    use_f0 = False          # Configuración F0 (False = no usar paralelización)
    f0_threshold = 3.0      # Umbral de pitch (valor por defecto)
    
    # Mostrar configuración
    print(f"\n📋 CONFIGURACIÓN DE PREPROCESAMIENTO:")
    print(f"   • Dataset de entrada: {dataset_folder}")
    print(f"   • Modelo: {model_name}")
    print(f"   • Sample rate: {sample_rate} Hz")
    print(f"   • Hilos de procesamiento: {num_threads}")
    print(f"   • Directorio de salida: {log_path}")
    print(f"   • Uso de F0: {use_f0}")
    print(f"   • Umbral F0: {f0_threshold}")
    
    # Verificar que existe el dataset
    if not os.path.exists(dataset_folder):
        print(f"❌ ERROR: No se encuentra el dataset en {dataset_folder}")
        print("   Asegúrate de haber ejecutado la celda de conversión MP3 a WAV primero")
    else:
        wav_files = [f for f in os.listdir(dataset_folder) if f.lower().endswith('.wav')]
        print(f"✅ Dataset encontrado: {len(wav_files)} archivos WAV")
        
        if len(wav_files) == 0:
            print("⚠️  ADVERTENCIA: No se encontraron archivos WAV en el dataset")
        else:
            print(f"   Primeros archivos: {wav_files[:3]}")
    
    # =============================================================================
    # 🚀 EJECUCIÓN DEL PREPROCESAMIENTO / PREPROCESSING EXECUTION
    # =============================================================================
    
    print(f"\n🚀 Iniciando preprocesamiento del dataset...")
    print("Este proceso puede tardar varios minutos dependiendo del tamaño del dataset")
    
    # Ejecutar la función preprocess_trainset
    try:
        preprocess_trainset(
            inp_root=dataset_folder,     # Directorio de entrada con archivos WAV
            sr=sample_rate,              # Sample rate
            n_p=num_threads,             # Número de procesos paralelos
            exp_dir=log_path,            # Directorio de experimento (salida)
            per=f0_threshold,            # Percentil para F0
            noparallel=use_f0            # Si usar paralelización (False = sí usar)
        )
        print("\n✅ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE!")
        print(f"📁 Archivos generados en: {log_path}")
        
        # Verificar archivos generados
        if os.path.exists(log_path):
            subdirs = [d for d in os.listdir(log_path) if os.path.isdir(os.path.join(log_path, d))]
            print(f"📂 Subdirectorios creados: {subdirs}")
            
            # Verificar contenido de cada subdirectorio
            for subdir in subdirs:
                subdir_path = os.path.join(log_path, subdir)
                files_count = len([f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))])
                print(f"   • {subdir}: {files_count} archivos")
        else:
            print("⚠️  Directorio de salida no encontrado, pero proceso completó sin errores")
            
    except Exception as e:
        print(f"\n❌ ERROR DURANTE EL PREPROCESAMIENTO:")
        print(f"   Tipo de error: {type(e).__name__}")
        print(f"   Mensaje: {str(e)}")
        # Mostrar traceback completo para debugging
        import traceback
        print(f"\n🔍 TRACEBACK COMPLETO:")
        traceback.print_exc()

finally:
    # Restaurar directorio original
    os.chdir(original_cwd)
    print(f"\n📂 Restaurado al directorio original: {os.getcwd()}")

📂 RVC Path: C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_proyect\RVC
📂 Directorio original: c:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect
📂 Cambiado a directorio RVC: C:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\RVC
📥 Importando función de preprocesamiento...
✅ Función preprocess_trainset importada exitosamente

📋 CONFIGURACIÓN DE PREPROCESAMIENTO:
   • Dataset de entrada: C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_dataset_wav
   • Modelo: My-Voice
   • Sample rate: 32000 Hz
   • Hilos de procesamiento: 2
   • Directorio de salida: C:/Users/jose_/Documents/samsung_ic/datasets/innobrand_proyect/RVC/logs/My-Voice
   • Uso de F0: False
   • Umbral F0: 3.0
✅ Dataset encontrado: 20 archivos WAV
   Primeros archivos: ['Audio 1.wav', 'Audio 10.wav', 'Audio 11.wav']

🚀 Iniciando preprocesamiento del dataset...
Este proceso puede tardar varios minutos dependiendo del tamaño del dataset
start preprocess
end preprocess

✅ PREPROCESAMIENTO C

In [7]:
import os
import sys
from IPython.display import clear_output

# Verificar dependencias antes de continuar
# Add RVC to Python path
rvc_path = os.path.join(curr_path, 'RVC')
if rvc_path not in sys.path:
    sys.path.append(rvc_path)

# Change to RVC directory for relative paths
original_cwd = os.getcwd()
os.chdir(rvc_path)

try:
    # Import the modified extraction functions
    print("\nImportando funciones de extracción...")
    from infer.modules.train.extract.extract_f0_print import extract_f0_print
    
    # Intentar importar fairseq, si falla usar solo F0
    try:
        from infer.modules.train.extract_feature_print import extract_feature_print
        fairseq_available = True
        print("✓ Funciones de extracción HuBERT disponibles")
    except ImportError as e:
        print(f"❌ Error importando HuBERT: {e}")
        print("Continuando solo con extracción F0...")
        fairseq_available = False
    
    f0method = "pm"  # Options: ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
    exp_dir_path = f"./logs/{model_name}"
    
    print(f"\nIniciando extracción usando funciones Python puras...")
    
    # Extract F0 features (esto siempre debería funcionar)
    print("Paso 1: Extrayendo características F0...")
    f0_success = extract_f0_print(
        exp_dir=exp_dir_path,
        n_p=2,  # Number of processes
        f0method=f0method
    )
    
    if f0_success:
        print("✓ Extracción F0 completada exitosamente!")
        
        # Extract HuBERT features (solo si fairseq está disponible)
        if fairseq_available:
            print("Paso 2: Extrayendo características HuBERT...")
            try:
                feature_success = extract_feature_print(
                    device_arg="cuda:0",
                    n_part=1,
                    i_part=0,
                    exp_dir=exp_dir_path,
                    version="v2",
                    is_half=True
                )
                
                if feature_success:
                    print("✓ Extracción HuBERT completada exitosamente!")
                    
                    # Check completion
                    log_file_path = f'{exp_dir_path}/extract_f0_feature.log'
                    try:
                        with open(log_file_path, 'r') as f:
                            log_content = f.read()
                            if 'all-feature-done' in log_content:
                                clear_output()
                                print("✓ Extracción F0 y características completada exitosamente!")
                            else:
                                print("Proceso completado pero marcador 'all-feature-done' no encontrado")
                    except FileNotFoundError:
                        print("Advertencia: Archivo de log no encontrado, pero extracción aparenta ser exitosa")
                else:
                    print("❌ Extracción HuBERT falló")
            except Exception as feature_error:
                print(f"❌ Error en extracción HuBERT: {feature_error}")
                print("Continuando solo con F0...")
        else:
            print("⚠️ Saltando extracción HuBERT (fairseq no disponible)")
            print("Solo se completó la extracción F0. Para entrenamiento completo, resuelve los problemas de fairseq.")
    else:
        print("❌ Extracción F0 falló")
        
except ImportError as import_error:
    print(f"❌ Error de importación: {import_error}")
    print("Algunas dependencias requeridas aún faltan.")
    print("Ejecuta la celda anterior para instalar dependencias.")
except Exception as e:
    print(f"Error durante extracción F0 y características: {e}")
    import traceback
    traceback.print_exc()
finally:
    # Restore original working directory
    os.chdir(original_cwd)

✓ Extracción F0 y características completada exitosamente!


In [8]:
import numpy as np
import faiss
import traceback
from sklearn.cluster import MiniBatchKMeans
from IPython.display import clear_output, display
from ipywidgets import Button

from RVC.configs import config

os.chdir(curr_path + '/RVC')

def train_index(exp_dir1, version19):
    """
    Construye un índice FAISS para búsqueda rápida de características durante la inferencia.
    
    Args:
        exp_dir1: Nombre del experimento/modelo
        version19: Versión del modelo ("v1" o "v2")
    
    Returns:
        Generator que yield mensajes de progreso
    """
    # Configurar directorios
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Seleccionar directorio de características según la versión
    # v1 usa características de 256 dimensiones, v2 usa 768 dimensiones
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    
    # Verificar que existan las características extraídas
    if not os.path.exists(feature_dir):
        return "¡Por favor, realiza primero la extracción de características!"
    
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "¡Por favor, realiza primero la extracción de características!"
    
    infos = []  # Lista para almacenar mensajes de progreso
    npys = []   # Lista para almacenar vectores de características
    
    # Cargar todos los archivos .npy de características
    print(f"Cargando {len(listdir_res)} archivos de características...")
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    
    # Concatenar todas las características en un solo array
    big_npy = np.concatenate(npys, 0)
    print(f"Características totales: {big_npy.shape}")
    
    # Mezclar aleatoriamente las características para mejor distribución
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    
    # Si hay más de 200k vectores, aplicar K-means para reducir a 10k centros
    # Esto reduce el tamaño del índice y mejora la velocidad de búsqueda
    if big_npy.shape[0] > 2e5:
        infos.append("Aplicando K-means: reduciendo %s vectores a 10k centros." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            # MiniBatchKMeans es más eficiente en memoria que KMeans regular
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,          # Reducir a 10k centros
                    verbose=True,              # Mostrar progreso
                    batch_size=256 * config.n_cpu,  # Tamaño de lote basado en CPUs
                    compute_labels=False,      # No necesitamos las etiquetas
                    init="random",             # Inicialización aleatoria
                )
                .fit(big_npy)
                .cluster_centers_             # Solo guardamos los centros
            )
            print(f"K-means completado. Nuevas dimensiones: {big_npy.shape}")
        except:
            info = traceback.format_exc()
            infos.append(f"Error en K-means: {info}")
            yield "\n".join(infos)

    # Guardar las características procesadas
    feature_file = "%s/total_fea.npy" % exp_dir
    np.save(feature_file, big_npy)
    print(f"Características guardadas en: {feature_file}")
    
    # Calcular parámetros para el índice FAISS
    # n_ivf: número de clusters invertidos (afecta velocidad vs precisión)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("Forma de características: %s, Clusters IVF: %s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    
    # Crear índice FAISS
    # IVF = Inverted File (búsqueda aproximada rápida)
    # Flat = búsqueda exacta dentro de cada cluster
    dimension = 256 if version19 == "v1" else 768
    index = faiss.index_factory(dimension, "IVF%s,Flat" % n_ivf)
    
    infos.append("Entrenando índice FAISS...")
    yield "\n".join(infos)
    
    # Configurar parámetros del índice
    index_ivf = faiss.extract_index_ivf(index)
    index_ivf.nprobe = 1  # Número de clusters a buscar (1 = más rápido)
    
    # Entrenar el índice con las características
    print("Entrenando índice FAISS...")
    index.train(big_npy)
    
    # Guardar índice entrenado (sin datos)
    trained_index_path = (
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )
    faiss.write_index(index, trained_index_path)
    print(f"Índice entrenado guardado: {trained_index_path}")

    infos.append("Agregando vectores al índice...")
    yield "\n".join(infos)
    
    # Agregar vectores al índice en lotes para eficiencia de memoria
    batch_size_add = 8192
    total_vectors = big_npy.shape[0]
    
    print(f"Agregando {total_vectors} vectores en lotes de {batch_size_add}...")
    for i in range(0, total_vectors, batch_size_add):
        batch_end = min(i + batch_size_add, total_vectors)
        index.add(big_npy[i:batch_end])
        if i % (batch_size_add * 4) == 0:  # Progreso cada 4 lotes
            print(f"Progreso: {i}/{total_vectors} vectores agregados")
    
    # Guardar índice completo (con datos)
    final_index_path = (
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )
    faiss.write_index(index, final_index_path)
    
    # Mensaje de éxito
    success_msg = (
        "✅ Índice construido exitosamente: added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )
    infos.append(success_msg)
    yield "\n".join(infos)
    
    print(f"\n🎉 ÍNDICE COMPLETADO:")
    print(f"📁 Archivo: {final_index_path}")
    print(f"📊 Vectores: {total_vectors}")
    print(f"🔍 Clusters IVF: {n_ivf}")
    print(f"📐 Dimensiones: {dimension}")

# Ejecutar construcción del índice
print("🚀 Iniciando construcción del índice FAISS...")
print("Este proceso puede tomar varios minutos dependiendo del tamaño del dataset.")

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line.lower() or 'agregando' in line.lower():
        clear_output()
        display(Button(description="✅ Índice Completado", button_style="success"))

Button(button_style='success', description='✅ Índice Completado', style=ButtonStyle())


🎉 ÍNDICE COMPLETADO:
📁 Archivo: logs/My-Voice/added_IVF1264_Flat_nprobe_1_My-Voice_v2.index
📊 Vectores: 49319
🔍 Clusters IVF: 1264
📐 Dimensiones: 768


In [ ]:
# =============================================================================
# 🖥️ DETECCIÓN Y CONFIGURACIÓN AUTOMÁTICA DE GPU / AUTO GPU DETECTION & CONFIG
# =============================================================================

import torch

print("🔍 Detectando configuración de GPU...")

# Verificar disponibilidad de CUDA y GPU
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
    
    print(f"✅ GPU detectada: {gpu_name}")
    print(f"📊 Memoria GPU: {gpu_memory:.1f} GB")
    print(f"🔢 Número de GPUs: {gpu_count}")
    
    # Configurar GPU según hardware disponible
    if gpu_count == 1:
        gpus = "0"  # Una sola GPU
        batch_size = 4 if gpu_memory >= 8 else 2  # Ajustar batch según memoria
    else:
        gpus = ",".join([str(i) for i in range(gpu_count)])  # Multi-GPU
        batch_size = 6 if gpu_memory >= 8 else 4
    
    print(f"⚙️ Configuración GPU: {gpus}")
    print(f"📦 Batch Size ajustado: {batch_size}")
    
else:
    print("❌ No se detectó GPU CUDA disponible")
    print("⚠️ ADVERTENCIA: RVC requiere GPU para entrenamiento eficiente")
    print("💡 Considera usar Google Colab o instalar CUDA")
    
    # Configuración fallback para CPU (no recomendado para entrenamiento)
    gpus = "0"  # Forzar a intentar GPU 0
    batch_size = 1
    
    print(f"🔧 Usando configuración básica: GPU={gpus}, Batch={batch_size}")

# Actualizar variables globales
batch_size = batch_size  # Sobrescribir batch_size anterior si existía

print(f"✅ Configuración GPU completada: {gpus}")
print(f"📋 Batch size final: {batch_size}")
print("=" * 60)

In [17]:
# =============================================================================
# ✅ VERIFICACIÓN FINAL SINGLE-GPU PARA RTX 4050 / FINAL SINGLE-GPU CHECK
# =============================================================================

import torch
import os

print("🔍 VERIFICACIÓN FINAL DE CONFIGURACIÓN SINGLE-GPU")
print("=" * 60)

# 1. VERIFICAR VARIABLES DE ENTORNO
print("1. Variables de entorno:")
env_vars = [
    'CUDA_VISIBLE_DEVICES', 'WORLD_SIZE', 'RANK', 'LOCAL_RANK',
    'NCCL_P2P_DISABLE', 'NCCL_IB_DISABLE'
]

for var in env_vars:
    value = os.environ.get(var, 'No configurado')
    status = "✅" if var in os.environ else "⚠️"
    print(f"   {status} {var}: {value}")

# 2. VERIFICAR PYTORCH Y GPU
print("\n2. Configuración PyTorch:")
if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(current_device)
    device_memory = torch.cuda.get_device_properties(current_device).total_memory / 1024**3
    
    print(f"   ✅ GPU activa: {current_device}")
    print(f"   ✅ Nombre: {device_name}")
    print(f"   ✅ Memoria: {device_memory:.1f} GB")
    print(f"   ✅ CuDNN: {torch.backends.cudnn.is_available()}")
    
    # Test de memoria disponible
    try:
        torch.cuda.empty_cache()
        memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
        memory_cached = torch.cuda.memory_reserved(0) / 1024**3
        memory_free = device_memory - memory_cached
        
        print(f"   📊 Memoria usada: {memory_allocated:.2f} GB")
        print(f"   📊 Memoria reservada: {memory_cached:.2f} GB")
        print(f"   📊 Memoria libre: {memory_free:.2f} GB")
        
        if memory_free >= 4.0:
            print("   ✅ Memoria suficiente para entrenamiento")
        else:
            print("   ⚠️ Memoria limitada - usar batch_size=1")
            
    except Exception as e:
        print(f"   ⚠️ Error verificando memoria: {e}")
else:
    print("   ❌ GPU no disponible")

# 3. VERIFICAR CONFIGURACIÓN RVC
print("\n3. Configuración RVC:")
config_items = [
    ('model_name', globals().get('model_name', 'No definido')),
    ('gpu_config', globals().get('gpu_config', 'No definido')),
    ('batch_size', globals().get('batch_size', 'No definido')),
    ('sample_rate', globals().get('sample_rate', 'No definido')),
    ('epochs', globals().get('epochs', 'No definido')),
    ('cache', globals().get('cache', 'No definido')),
    ('OV2', globals().get('OV2', 'No definido')),
]

for name, value in config_items:
    print(f"   📋 {name}: {value}")

# 4. RECOMENDACIONES FINALES
print("\n4. Recomendaciones finales para RTX 4050:")
print("   🎯 Configuración ÓPTIMA detectada:")

# Verificar y ajustar configuración final
final_batch_size = globals().get('batch_size', 2)
final_cache = globals().get('cache', False)
final_gpu_config = globals().get('gpu_config', '0')

if final_batch_size > 4:
    print("   ⚠️ Batch size muy alto, se recomienda ≤2 para RTX 4050")
    final_batch_size = 2
else:
    print(f"   ✅ Batch size: {final_batch_size} (apropiado)")

if final_cache:
    print("   ⚠️ Caché habilitado puede causar problemas de memoria")
    final_cache = False
else:
    print(f"   ✅ Caché GPU: {final_cache} (correcto)")

if final_gpu_config != "0":
    print("   ⚠️ GPU config debe ser '0' para single-GPU")
    final_gpu_config = "0"
else:
    print(f"   ✅ GPU config: {final_gpu_config} (correcto)")

# Actualizar variables globales con valores corregidos
globals()['batch_size'] = final_batch_size
globals()['cache'] = final_cache
globals()['gpu_config'] = final_gpu_config

print("\n🎉 VERIFICACIÓN COMPLETADA")
print("✅ Sistema listo para entrenamiento single-GPU en RTX 4050")
print("💡 El entrenamiento debería ejecutarse sin errores de distributed training")
print("=" * 60)

🔍 VERIFICACIÓN FINAL DE CONFIGURACIÓN SINGLE-GPU
1. Variables de entorno:
   ✅ CUDA_VISIBLE_DEVICES: 0
   ✅ WORLD_SIZE: 1
   ✅ RANK: 0
   ✅ LOCAL_RANK: 0
   ✅ NCCL_P2P_DISABLE: 1
   ✅ NCCL_IB_DISABLE: 1

2. Configuración PyTorch:
   ✅ GPU activa: 0
   ✅ Nombre: NVIDIA GeForce RTX 4050 Laptop GPU
   ✅ Memoria: 6.0 GB
   ✅ CuDNN: True
   📊 Memoria usada: 0.00 GB
   📊 Memoria reservada: 0.00 GB
   📊 Memoria libre: 6.00 GB
   ✅ Memoria suficiente para entrenamiento

3. Configuración RVC:
   📋 model_name: My-Voice
   📋 gpu_config: 0
   📋 batch_size: 1
   📋 sample_rate: 32k
   📋 epochs: 300
   📋 cache: False
   📋 OV2: False

4. Recomendaciones finales para RTX 4050:
   🎯 Configuración ÓPTIMA detectada:
   ✅ Batch size: 1 (apropiado)
   ✅ Caché GPU: False (correcto)
   ✅ GPU config: 0 (correcto)

🎉 VERIFICACIÓN COMPLETADA
✅ Sistema listo para entrenamiento single-GPU en RTX 4050
💡 El entrenamiento debería ejecutarse sin errores de distributed training


In [19]:
# =============================================================================
# 🎓 ENTRENAMIENTO DE MODELO RVC / RVC MODEL TRAINING
# =============================================================================
# Este código implementa el entrenamiento completo de un modelo de conversión de voz
# usando la arquitectura RVC (Retrieval-based Voice Conversion)

#@title  **TRAIN YOUR MODEL** - You can also resume training here!

# =============================================================================
# 📦 IMPORTACIONES NECESARIAS / REQUIRED IMPORTS
# =============================================================================
from random import shuffle      # Para mezclar datos aleatoriamente
import json                    # Para manejar archivos de configuración JSON
import os                      # Para operaciones del sistema de archivos
import pathlib                 # Para manejo moderno de rutas de archivos
from subprocess import Popen, PIPE, STDOUT  # Para ejecutar procesos externos
import torch                   # Para verificación de GPU

now_dir = os.getcwd()  # Obtener directorio actual de trabajo

# =============================================================================
# ⚙️ CONFIGURACIÓN DE PARÁMETROS DE ENTRENAMIENTO / TRAINING PARAMETERS CONFIG
# =============================================================================

# 🔄 Parámetros principales de entrenamiento
# model_name = 'My-Voice'  # Nombre del modelo (definido en celdas anteriores)

# 💾 Frecuencia de guardado y duración del entrenamiento
save_frequency = 25    # @param {type:"slider", min:5, max:50, step:5}
                      # Guardar checkpoint cada 25 epochs
epochs = 300          # @param {type:"slider", min:10, max:2000, step:10} Se pueden usar 500
                      # Número total de epochs de entrenamiento

# 🚀 Configuraciones avanzadas
OV2 = False           # @param {type:"boolean"} 
                     # Usar modelo OV2 Super (versión mejorada con mejor calidad)

open_tensorboard = True  # @param{type:"boolean"}
                        # Abrir TensorBoard para monitoreo en tiempo real

# =============================================================================
# 🖥️ CONFIGURACIÓN FORZADA SINGLE-GPU PARA RTX 4050 / FORCED SINGLE-GPU CONFIG
# =============================================================================

import os

# FORZAR CONFIGURACIÓN SINGLE-GPU para evitar errores distributed training
print("🔧 CONFIGURANDO SINGLE-GPU PARA RTX 4050 LAPTOP")
print("=" * 60)

# 1. CONFIGURAR VARIABLES DE ENTORNO para single-GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'        # Solo GPU 0 visible
os.environ['WORLD_SIZE'] = '1'                  # Un solo proceso
os.environ['RANK'] = '0'                        # Rank principal
os.environ['LOCAL_RANK'] = '0'                  # Rank local 0
os.environ['MASTER_ADDR'] = 'localhost'         # Dirección local
os.environ['MASTER_PORT'] = '12355'             # Puerto libre

# 2. DESHABILITAR backends distribuidos problemáticos
os.environ['NCCL_P2P_DISABLE'] = '1'           # Deshabilitar P2P
os.environ['NCCL_IB_DISABLE'] = '1'            # Deshabilitar InfiniBand

print("✅ Variables de entorno configuradas para single-GPU")

# 3. CONFIGURAR PyTorch para single-GPU
if torch.cuda.is_available():
    # Forzar uso de GPU 0 únicamente
    torch.cuda.set_device(0)
    
    # Configuración optimizada para RTX 4050
    torch.backends.cudnn.benchmark = False      # Más estable para batch pequeño
    torch.backends.cudnn.deterministic = True   # Reproducible
    
    # Obtener información de la GPU
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    # CONFIGURACIÓN ESPECÍFICA PARA RTX 4050 (6GB VRAM)
    gpu_config = "0"                            # Solo GPU 0
    
    # Batch size optimizado para 6GB VRAM
    if gpu_memory >= 8:
        batch_size = 4                          # Para GPUs con >8GB
    elif gpu_memory >= 6:
        batch_size = 2                          # Para RTX 4050 (6GB)
    else:
        batch_size = 1                          # Para GPUs con <6GB
    
    print(f"🖥️  GPU detectada: {gpu_name}")
    print(f"💾 Memoria GPU: {gpu_memory:.1f} GB")
    print(f"🔢 GPUs disponibles: {gpu_count} (usando solo GPU 0)")
    print(f"⚙️  Configuración GPU: {gpu_config}")
    print(f"📦 Batch Size optimizado: {batch_size}")
    
    # Test de funcionamiento single-GPU
    try:
        test_tensor = torch.randn(2, 100).cuda()
        result = test_tensor * 2
        print("✅ Test single-GPU exitoso")
        del test_tensor, result
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ Error en test GPU: {e}")
    
else:
    print("❌ ADVERTENCIA: No se detectó GPU CUDA")
    print("💡 Para RTX 4050, asegúrate de:")
    print("   1. Tener drivers NVIDIA actualizados")
    print("   2. PyTorch con CUDA instalado:")
    print("      pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
    
    # Configuración fallback
    gpu_config = "0"
    batch_size = 1
    print(f"🔧 Usando configuración fallback...")

# 4. CONFIGURACIONES ADICIONALES para RTX 4050
cache = False                                   # No usar caché GPU (ahorra VRAM)
mixed_precision = True                          # Usar FP16 (reduce memoria 50%)
use_distributed = False                         # NUNCA usar entrenamiento distribuido

print(f"💾 Caché GPU: {cache} (deshabilitado para ahorrar VRAM)")
print(f"🔄 Mixed Precision: {mixed_precision} (FP16 para ahorrar memoria)")
print(f"🚫 Distributed Training: {use_distributed} (deshabilitado)")
print("=" * 60)

# 🎵 Configuración de audio
sample_rate = '32k'  # Frecuencia de muestreo (32kHz es óptimo para la mayoría de casos)

# 🤖 Selección automática de modelos preentrenados según configuración
if OV2:
    # Modelos OV2 Super - versión mejorada con mejor calidad de audio
    G_file = f'assets/pretrained_v2/f0Ov2Super{sample_rate}G.pth'  # Generador
    D_file = f'assets/pretrained_v2/f0Ov2Super{sample_rate}D.pth'  # Discriminador
else:
    # Modelos estándar v2
    G_file = f'assets/pretrained_v2/f0G{sample_rate}.pth'
    D_file = f'assets/pretrained_v2/f0D{sample_rate}.pth'

# =============================================================================
# 🏗️ FUNCIÓN PRINCIPAL DE ENTRENAMIENTO / MAIN TRAINING FUNCTION
# =============================================================================
def click_train(
    exp_dir1,           # Nombre del experimento/modelo
    sr2,                # Sample rate (frecuencia de muestreo)
    if_f0_3,           # Si usar información F0 (pitch/tono)
    spk_id5,           # ID del hablante (0 para un solo hablante)
    save_epoch10,      # Frecuencia de guardado de checkpoints
    total_epoch11,     # Número total de epochs
    batch_size12,      # Tamaño del batch/lote
    if_save_latest13,  # Si guardar siempre el modelo más reciente
    pretrained_G14,    # Ruta del generador preentrenado
    pretrained_D15,    # Ruta del discriminador preentrenado
    gpus16,           # IDs de GPUs a usar (0 = primera GPU)
    if_cache_gpu17,   # Si usar caché en GPU para acelerar entrenamiento
    if_save_every_weights18,  # Si guardar pesos en cada checkpoint
    version19,        # Versión del modelo ("v1" o "v2")
):
    """
    Función principal que ejecuta el entrenamiento del modelo RVC.
    
    Proceso:
    1. Generar lista de archivos de entrenamiento (filelist)
    2. Configurar parámetros del modelo
    3. Ejecutar script de entrenamiento
    4. Monitorear progreso en tiempo real
    """
    
    # =========================================================================
    # 📁 CONFIGURACIÓN DE DIRECTORIOS / DIRECTORY SETUP
    # =========================================================================
    
    # Crear directorio del experimento
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Definir rutas de directorios de datos
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)      # Audio original (ground truth)
    
    # Seleccionar directorio de características según versión del modelo
    # v1 usa 256 dimensiones, v2 usa 768 dimensiones
    feature_dir = (
        "%s/3_feature256" % (exp_dir)  # Para modelo v1
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)  # Para modelo v2
    )
    
    # =========================================================================
    # 🎯 IDENTIFICACIÓN DE ARCHIVOS VÁLIDOS / VALID FILES IDENTIFICATION
    # =========================================================================
    
    if if_f0_3:  # Si se usa información de pitch/tono
        # Directorios adicionales para información F0
        f0_dir = "%s/2a_f0" % (exp_dir)        # Archivos F0 básicos
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)  # Archivos F0 NSF (mejorados)
        
        # Encontrar archivos que tienen TODOS los componentes necesarios
        # Usando intersección de conjuntos para asegurar completitud
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) &
            set([name.split(".")[0] for name in os.listdir(feature_dir)]) &
            set([name.split(".")[0] for name in os.listdir(f0_dir)]) &
            set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:  # Sin información de pitch
        # Solo necesitamos audio original y características
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    
    # =========================================================================
    # 📝 GENERACIÓN DE LISTA DE ENTRENAMIENTO / TRAINING LIST GENERATION
    # =========================================================================
    
    opt = []  # Lista que contendrá todas las entradas de entrenamiento
    
    # Crear entrada para cada archivo válido
    for name in names:
        if if_f0_3:  # Con información de pitch
            # Formato: audio|características|f0|f0nsf|speaker_id
            # Cada "|" separa un componente diferente del entrenamiento
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"), name,    # Audio original
                    feature_dir.replace("\\", "\\\\"), name,    # Características HuBERT
                    f0_dir.replace("\\", "\\\\"), name,         # Información F0
                    f0nsf_dir.replace("\\", "\\\\"), name,      # Información F0 NSF
                    spk_id5,                                     # ID del hablante
                )
            )
        else:  # Sin información de pitch
            # Formato simplificado: audio|características|speaker_id
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"), name,
                    feature_dir.replace("\\", "\\\\"), name,
                    spk_id5,
                )
            )
    
    # =========================================================================
    # 🔇 AGREGAR ARCHIVOS "MUTE" PARA ESTABILIDAD / ADD MUTE FILES FOR STABILITY
    # =========================================================================
    
    # Determinar dimensión de características según versión
    fea_dim = 256 if version19 == "v1" else 768
    
    # Agregar archivos "mute" (silencio) para estabilizar el entrenamiento
    # Estos archivos ayudan a que el modelo no se sobreajuste
    if if_f0_3:
        for _ in range(2):  # Agregar 2 entradas mute
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    
    # Mezclar la lista para entrenamiento aleatorio (mejora la convergencia)
    shuffle(opt)
    
    # Guardar lista de archivos de entrenamiento
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # =========================================================================
    # 📊 INFORMACIÓN DE CONFIGURACIÓN / CONFIGURATION INFO
    # =========================================================================
    
    # Mostrar información de la configuración actual
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    
    # =========================================================================
    # ⚙️ CONFIGURACIÓN DEL MODELO / MODEL CONFIGURATION
    # =========================================================================
    
    # Seleccionar archivo de configuración según versión y sample rate
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    
    # Copiar configuración al directorio del experimento
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    # =========================================================================
    # 🚀 CONSTRUCCIÓN Y EJECUCIÓN DEL COMANDO DE ENTRENAMIENTO / TRAINING COMMAND
    # =========================================================================
    
    # Construir comando de entrenamiento con todos los parámetros
    cmd = (
        'python infer/modules/train/train.py '  # Script principal de entrenamiento
        '-e "%s" '      # Directorio del experimento
        '-sr %s '       # Sample rate
        '-f0 %s '       # Usar F0 (1=sí, 0=no)
        '-bs %s '       # Batch size
        '-g %s '        # GPUs a usar
        '-te %s '       # Total epochs
        '-se %s '       # Save epochs (frecuencia de guardado)
        '%s '           # Generador preentrenado (opcional)
        '%s '           # Discriminador preentrenado (opcional)
        '-l %s '        # Save latest (guardar último modelo)
        '-c %s '        # Cache GPU
        '-sw %s '       # Save weights (guardar pesos)
        '-v %s'         # Versión del modelo
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    
    # =========================================================================
    # 📺 EJECUCIÓN CON MONITOREO EN TIEMPO REAL / REAL-TIME MONITORING
    # =========================================================================
    
    # Ejecutar comando con captura de salida en tiempo real
    # PIPE permite capturar tanto stdout como stderr
    p = Popen(cmd, shell=True, cwd=now_dir, 
              stdout=PIPE, stderr=STDOUT, 
              bufsize=1, universal_newlines=True)

    # Mostrar salida del entrenamiento en tiempo real
    # Esto permite ver el progreso, pérdidas, y posibles errores
    for line in p.stdout:
        print(line.strip())

    # Esperar a que termine el proceso de entrenamiento
    p.wait()
    
    # Mensaje de finalización (traducido al español)
    return "Entrenamiento finalizado, puede revisar los logs de entrenamiento en la consola o en train.log en la carpeta del experimento"

# =============================================================================
# 📊 CONFIGURACIÓN DE TENSORBOARD PARA MONITOREO / TENSORBOARD MONITORING SETUP
# =============================================================================

if open_tensorboard:
    # TensorBoard permite visualizar:
    # - Pérdidas del generador y discriminador en tiempo real
    # - Espectrogramas de audio original vs generado
    # - Métricas de entrenamiento y progreso
    # - Gráficas de learning rate y gradientes
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888

# =============================================================================
# 🎯 EJECUCIÓN DEL ENTRENAMIENTO SINGLE-GPU / SINGLE-GPU TRAINING EXECUTION
# =============================================================================

# CONFIGURACIÓN FINAL OPTIMIZADA PARA RTX 4050 SINGLE-GPU
print("🚀 INICIANDO ENTRENAMIENTO SINGLE-GPU OPTIMIZADO PARA RTX 4050...")
print("=" * 70)

# Verificar que las variables estén definidas
if "cache" not in locals():
    cache = False  # Deshabilitado para RTX 4050 (ahorra VRAM)

if "mixed_precision" not in locals():
    mixed_precision = True  # Habilitar FP16 para ahorrar memoria

print(f"📋 CONFIGURACIÓN FINAL SINGLE-GPU:")
print(f"   • Modelo: {model_name}")
print(f"   • Sample Rate: {sample_rate}")
print(f"   • Epochs: {epochs}")
print(f"   • Batch Size: {batch_size} (optimizado para 6GB VRAM)")
print(f"   • GPU Config: {gpu_config} (solo GPU 0)")
print(f"   • Guardar cada: {save_frequency} epochs")
print(f"   • Usar F0: Sí (pitch tracking habilitado)")
print(f"   • Versión: v2 (recomendado)")
print(f"   • Modelos OV2: {'Sí' if OV2 else 'No'} (False recomendado para RTX 4050)")
print(f"   • Caché GPU: {cache} (deshabilitado para ahorrar VRAM)")
print(f"   • Mixed Precision: {mixed_precision} (FP16 para eficiencia)")
print(f"   • Distributed: False (single-GPU forzado)")
print("=" * 70)

# VERIFICACIÓN FINAL DE CONFIGURACIÓN SINGLE-GPU
print("🔍 Verificación final de configuración single-GPU:")
if gpu_config == "0":
    print("✅ GPU Config correcta: solo GPU 0")
else:
    print("⚠️ Corrigiendo GPU Config a single-GPU...")
    gpu_config = "0"

if batch_size <= 4:
    print(f"✅ Batch Size seguro para RTX 4050: {batch_size}")
else:
    print("⚠️ Batch Size muy alto para RTX 4050, reduciendo...")
    batch_size = 2

if not cache:
    print("✅ Caché GPU deshabilitado (correcto para 6GB VRAM)")
else:
    print("⚠️ Corrigiendo caché GPU...")
    cache = False

print("=" * 70)

# 🚀 EJECUTAR ENTRENAMIENTO con configuración single-GPU optimizada
training_log = click_train(
    model_name,      # Nombre del modelo
    sample_rate,     # '32k' - frecuencia de muestreo optimizada
    True,           # if_f0_3 - usar información de pitch/tono (siempre True para calidad)
    0,              # spk_id5 - ID del hablante (0 para single-speaker)
    save_frequency, # Guardar checkpoint cada X epochs (25 recomendado)
    epochs,         # Número total de epochs (300 balanceado)
    batch_size,     # Tamaño del lote optimizado para RTX 4050
    True,           # if_save_latest13 - siempre guardar modelo más reciente
    G_file,         # Generador preentrenado (v2 estándar)
    D_file,         # Discriminador preentrenado (v2 estándar)
    gpu_config,     # "0" - configuración single-GPU forzada
    cache,          # False - sin caché para ahorrar VRAM
    True,           # if_save_every_weights18 - guardar todos los checkpoints
    'v2',           # version19 - usar modelo v2 (mejor calidad)
)

# Mostrar resultado final del entrenamiento
print("\n🎉 RESULTADO DEL ENTRENAMIENTO:")
print("=" * 70)
print(training_log)
print("=" * 70)

# INFORMACIÓN POST-ENTRENAMIENTO
print("\n📋 INFORMACIÓN POST-ENTRENAMIENTO:")
print("📁 Archivos generados en: ./logs/{}/".format(model_name))
print("📊 Monitoreo TensorBoard: http://localhost:8888")
print("💾 Checkpoints guardados cada {} epochs".format(save_frequency))
print("🎯 Modelo final: ./logs/{}/G_{}.pth".format(model_name, epochs))
print("🎵 Para inferencia, usa el archivo .pth generado")
print("=" * 70)

🔧 CONFIGURANDO SINGLE-GPU PARA RTX 4050 LAPTOP
✅ Variables de entorno configuradas para single-GPU
🖥️  GPU detectada: NVIDIA GeForce RTX 4050 Laptop GPU
💾 Memoria GPU: 6.0 GB
🔢 GPUs disponibles: 1 (usando solo GPU 0)
⚙️  Configuración GPU: 0
📦 Batch Size optimizado: 1
✅ Test single-GPU exitoso
💾 Caché GPU: False (deshabilitado para ahorrar VRAM)
🔄 Mixed Precision: True (FP16 para ahorrar memoria)
🚫 Distributed Training: False (deshabilitado)
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 8888 (pid 16952), started 8 days, 0:10:41 ago. (Use '!kill 16952' to kill it.)

🚀 INICIANDO ENTRENAMIENTO SINGLE-GPU OPTIMIZADO PARA RTX 4050...
📋 CONFIGURACIÓN FINAL SINGLE-GPU:
   • Modelo: My-Voice
   • Sample Rate: 32k
   • Epochs: 300
   • Batch Size: 1 (optimizado para 6GB VRAM)
   • GPU Config: 0 (solo GPU 0)
   • Guardar cada: 25 epochs
   • Usar F0: Sí (pitch tracking habilitado)
   • Versión: v2 (recomendado)
   • Modelos OV2: No (False recomendado para RTX 4050)
   • Caché GPU: False (deshabilitado para ahorrar VRAM)
   • Mixed Precision: True (FP16 para eficiencia)
   • Distributed: False (single-GPU forzado)
🔍 Verificación final de configuración single-GPU:
✅ GPU Config correcta: solo GPU 0
✅ Batch Size seguro para RTX 4050: 1
✅ Caché GPU deshabilitado (correcto para 6GB VRAM)
Write filelist done
Use gpus: 0
ðŸ”§ PARCHE SINGLE-GPU APLICADO - RTX 4050 MODE
ðŸ”§ PARCHE SINGLE-GPU APLICADO - RTX 4050 MODE
ðŸ”§ PARCHE SINGLE-GPU APLICADO - RTX 4050 MODE
INFO:My-Voice:{'data': {'filter_length': 1024, 'hop_length': 320, 'max_wav_value': 32768.0, 'mel_fmax':

In [18]:
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [30]:
# 🔧 CONFIGURACIÓN TENSORBOARD CORREGIDA
# Los comandos mágicos %load_ext y %tensorboard DEBEN ejecutarse en celdas separadas del notebook

print("🔧 CONFIGURANDO TENSORBOARD...")

# Verificar si TensorBoard está disponible
try:
    import tensorboard
    print("✅ TensorBoard disponible")
except ImportError:
    print("❌ TensorBoard no instalado, instalando...")
    import subprocess
    subprocess.run(["pip", "install", "tensorboard"])
    import tensorboard
    print("✅ TensorBoard instalado exitosamente")

# Configurar directorio de logs para TensorBoard
tensorboard_logdir = f"./RVC/logs/{model_name}"
print(f"📊 Directorio de logs: {tensorboard_logdir}")

# Verificar que existe el directorio de logs
if os.path.exists(tensorboard_logdir):
    print("✅ Directorio de logs encontrado")
else:
    print("⚠️ Directorio de logs no existe aún - se creará durante el entrenamiento")

print("📋 Para iniciar TensorBoard manualmente:")
print("1. Ejecuta en terminal: tensorboard --logdir=./RVC/logs --port=8888")
print("2. O ejecuta la siguiente celda que contiene los comandos mágicos")
print("3. Abre http://localhost:8888 en tu navegador")
print("✅ TensorBoard configurado - listo para entrenamiento")

🔧 CONFIGURANDO TENSORBOARD...
✅ TensorBoard disponible
📊 Directorio de logs: ./RVC/logs/My-Voice
⚠️ Directorio de logs no existe aún - se creará durante el entrenamiento
📋 Para iniciar TensorBoard manualmente:
1. Ejecuta en terminal: tensorboard --logdir=./RVC/logs --port=8888
2. O ejecuta la siguiente celda que contiene los comandos mágicos
3. Abre http://localhost:8888 en tu navegador
✅ TensorBoard configurado - listo para entrenamiento


In [31]:
# 📊 INICIAR TENSORBOARD (Comandos mágicos en celda separada)
# EJECUTA ESTA CELDA ANTES DEL ENTRENAMIENTO

%load_ext tensorboard
%tensorboard --logdir ./RVC/logs --port 8888

print("✅ TensorBoard iniciado en: http://localhost:8888")
print("🔥 Abre esa URL en tu navegador para monitorear el entrenamiento")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


✅ TensorBoard iniciado en: http://localhost:8888
🔥 Abre esa URL en tu navegador para monitorear el entrenamiento


In [ ]:
# 🎯 ENTRENAR MODELO RVC CON TODAS LAS CORRECCIONES APLICADAS
# ========================================================================
# ✅ CORRECCIONES IMPLEMENTADAS:
# 1. Single-GPU training sin distributed/multiprocessing (elimina error libuv)
# 2. TensorBoard programático sin comandos mágicos problemáticos
# 3. Configuración optimizada para RTX 4050 laptop
# 4. Rutas simplificadas - ya estamos en el directorio RVC
# 5. Ambiente Python correcto - usando sys.executable del notebook
# 6. Manejo de eval_interval faltante en configuración HParams
# ========================================================================

import os
import sys
import subprocess

# Definir variables faltantes usando los valores de variables del kernel
final_batch_size = globals().get('batch_size', 2)
final_cache = globals().get('cache', False)
use_f0 = True  # Para entrenamiento de voz humana siempre usar F0

print("🚀 INICIANDO ENTRENAMIENTO RVC - CONFIGURACIÓN SINGLE-GPU RTX 4050")
print("="*70)
print(f"📁 Modelo: {model_name}")
print(f"🎵 Sample Rate: {sample_rate}")
print(f"🔢 Epochs: {epochs}")
print(f"📦 Batch Size: {final_batch_size}")
print(f"🎛️ F0: {'Activado' if use_f0 else 'Desactivado'}")
print(f"💾 Cache GPU: {'Activado' if final_cache else 'Desactivado'}")
print(f"📊 TensorBoard: {'Activado' if open_tensorboard else 'Desactivado'}")
print("="*70)

# Verificar ambiente Python
python_executable = sys.executable
print(f"🐍 Python ejecutable: {python_executable}")
print(f"🗂️ Python version: {sys.version}")
print(f"📦 Python path: {sys.prefix}")

# Verificar directorio actual - debemos estar en RVC
current_dir = os.getcwd()
print(f"📂 Directorio actual: {current_dir}")

# Verificar que estamos en el directorio RVC correcto
required_items = ["infer", "assets", "logs"]
missing_items = [item for item in required_items if not os.path.exists(item)]

if missing_items:
    print(f"❌ Faltan elementos RVC: {missing_items}")
    print("📁 Contenido del directorio actual:")
    for item in os.listdir(current_dir):
        print(f"   • {item}")
    raise FileNotFoundError(f"Directorio RVC incompleto. Faltan: {missing_items}")
else:
    print("✅ Directorio RVC confirmado - todos los elementos necesarios presentes")

# Verificar script de entrenamiento modificado
train_script_path = "infer/modules/train/train.py"
if os.path.exists(train_script_path):
    with open(train_script_path, 'r', encoding='utf-8') as f:
        content = f.read()
        if "🔧 PARCHE SINGLE-GPU APLICADO" in content and "run_single_gpu" in content:
            print("✅ Script de entrenamiento modificado correctamente para single-GPU")
            
            # Verificar corrección específica de eval_interval
            if "getattr(hps.train, 'eval_interval'" in content:
                print("✅ Corrección eval_interval aplicada correctamente")
            else:
                print("⚠️ Corrección eval_interval faltante - podría causar errores")
        else:
            print("⚠️ Script de entrenamiento no modificado - usando versión original")
else:
    print(f"❌ Script de entrenamiento no encontrado: {train_script_path}")

# Verificar disponibilidad de dependencias en el ambiente actual
try:
    import torch
    print(f"✅ PyTorch disponible: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"✅ CUDA disponible: {torch.version.cuda}")
        
        # Información detallada de GPU para debugging
        device = torch.cuda.current_device()
        gpu_name = torch.cuda.get_device_name(device)
        memory_total = torch.cuda.get_device_properties(device).total_memory / 1024**3
        print(f"✅ GPU activa: {gpu_name} ({memory_total:.1f}GB)")
    else:
        print("⚠️ CUDA no disponible en este ambiente")
except ImportError as e:
    print(f"❌ Error importando PyTorch: {e}")

# Verificar dataset y archivos de entrenamiento
model_logs_path = f"./logs/{model_name}"
if os.path.exists(model_logs_path):
    filelist_path = f"{model_logs_path}/filelist.txt"
    if os.path.exists(filelist_path):
        with open(filelist_path, 'r') as f:
            lines = f.readlines()
            print(f"✅ Filelist encontrado: {len(lines)} archivos de entrenamiento")
    else:
        print("⚠️ Filelist no encontrado - podría necesitar regeneración")
    
    # Verificar directorios de datos necesarios
    data_dirs = ["0_gt_wavs", "2a_f0", "2b-f0nsf", "3_feature768"]
    for data_dir in data_dirs:
        dir_path = f"{model_logs_path}/{data_dir}"
        if os.path.exists(dir_path):
            file_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
            print(f"✅ {data_dir}: {file_count} archivos")
        else:
            print(f"❌ {data_dir}: directorio faltante")
else:
    print(f"❌ Directorio del modelo no encontrado: {model_logs_path}")

# Ejecutar entrenamiento directamente desde el directorio RVC actual
try:
    print(f"✅ Ejecutando desde directorio RVC: {os.getcwd()}")
    
    # Configurar variables de entorno para single-GPU
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = "0"
    env["WORLD_SIZE"] = "1"
    env["RANK"] = "0"
    env["LOCAL_RANK"] = "0"
    
    # Limpiar caché CUDA antes de iniciar
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("✅ Caché CUDA limpiado antes del entrenamiento")
    except:
        pass
    
    # Construir comando de entrenamiento con el Python correcto del ambiente
    train_command = [
        python_executable,  # Usar el mismo Python del notebook
        "infer/modules/train/train.py",
        "-e", model_name,
        "-sr", sample_rate,
        "-f0", "1" if use_f0 else "0",
        "-bs", str(final_batch_size),
        "-te", str(epochs),
        "-se", str(save_frequency),
        "-pg", "./assets/pretrained_v2/f0G32k.pth",
        "-pd", "./assets/pretrained_v2/f0D32k.pth",
        "-l", "1" if final_cache else "0",
        "-c", "0",  # single-GPU
        "-sw", "1",  # save weights
        "-v", "v2"
    ]
    
    print(f"🎯 Comando de entrenamiento:")
    print(" ".join(train_command))
    print("🔥 INICIANDO ENTRENAMIENTO...")
    print("📊 Monitorea el progreso en TensorBoard: http://localhost:8888")
    print("⏱️ El entrenamiento puede tomar varias horas dependiendo del dataset y epochs")
    print("🔄 Ejecutando con variables de entorno configuradas para single-GPU...")
    
    # Ejecutar entrenamiento con el ambiente correcto
    result = subprocess.run(
        train_command, 
        capture_output=True, 
        text=True, 
        env=env,  # Pasar variables de entorno configuradas
        cwd=current_dir  # Asegurar directorio de trabajo correcto
    )
    
    if result.returncode == 0:
        print("✅ ENTRENAMIENTO COMPLETADO CON ÉXITO!")
        print("📁 Modelos guardados en: ./logs/")
        print("🎵 Listo para usar en inferencia!")
        
        # Mostrar algunas líneas finales del log si está disponible
        if result.stdout:
            print("\n📋 ÚLTIMAS LÍNEAS DEL LOG:")
            stdout_lines = result.stdout.strip().split('\n')
            for line in stdout_lines[-10:]:  # Últimas 10 líneas
                print(f"   {line}")
    else:
        print("❌ ERROR EN EL ENTRENAMIENTO:")
        print(f"📥 Return code: {result.returncode}")
        
        if result.stdout:
            print("\n📤 STDOUT:")
            stdout_lines = result.stdout.strip().split('\n')
            for line in stdout_lines[-25:]:  # Últimas 25 líneas del stdout
                print(f"   {line}")
        
        if result.stderr:
            print("\n🚨 STDERR:")
            stderr_lines = result.stderr.strip().split('\n')
            for line in stderr_lines[-15:]:  # Últimas 15 líneas del stderr
                print(f"   {line}")
        
        # Análisis específico de errores conocidos
        full_output = (result.stdout or "") + (result.stderr or "")
        
        if "use_libuv was requested but PyTorch was build without libuv support" in full_output:
            print("\n🔧 ERROR DETECTADO: libuv")
            print("💡 SOLUCIÓN: Verificar que el script train.py esté modificado para single-GPU")
        
        if "No module named" in full_output:
            print("\n🔧 ERROR DETECTADO: Módulo faltante")
            print("💡 SOLUCIÓN: Instalar dependencias en el ambiente actual")
        
        if "CUDA" in full_output and "not available" in full_output:
            print("\n🔧 ERROR DETECTADO: CUDA no disponible")
            print("💡 SOLUCIÓN: Verificar instalación de PyTorch con CUDA")
            
        if "AttributeError" in full_output and "eval_interval" in full_output:
            print("\n🔧 ERROR DETECTADO: eval_interval faltante")
            print("💡 SOLUCIÓN: Script train.py necesita corrección adicional")
            
        if "FileNotFoundError" in full_output:
            print("\n🔧 ERROR DETECTADO: Archivo no encontrado")
            print("💡 SOLUCIÓN: Verificar que todos los archivos de preprocesamiento estén presentes")
            
        if "RuntimeError" in full_output and ("CUDA" in full_output or "GPU" in full_output):
            print("\n🔧 ERROR DETECTADO: Error de GPU/CUDA")
            print("💡 SOLUCIÓN: Verificar memoria GPU disponible o reducir batch_size")
        
except KeyboardInterrupt:
    print("⏹️ Entrenamiento interrumpido por el usuario")
except Exception as e:
    print(f"❌ Error durante el entrenamiento: {e}")
    import traceback
    traceback.print_exc()

print("🏁 Proceso de entrenamiento finalizado")

🚀 INICIANDO ENTRENAMIENTO RVC - CONFIGURACIÓN SINGLE-GPU RTX 4050
📁 Modelo: My-Voice
🎵 Sample Rate: 32k
🔢 Epochs: 300
📦 Batch Size: 1
🎛️ F0: Activado
💾 Cache GPU: Desactivado
📊 TensorBoard: Activado
🐍 Python ejecutable: c:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\.venv\Scripts\python.exe
🗂️ Python version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
📦 Python path: c:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\.venv
📂 Directorio actual: C:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\RVC
✅ Directorio RVC confirmado - todos los elementos necesarios presentes
✅ Script de entrenamiento modificado correctamente para single-GPU
✅ PyTorch disponible: 2.4.1+cu121
✅ CUDA disponible: 12.1
✅ Ejecutando desde directorio RVC: C:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\RVC
🎯 Comando de entrenamiento:
c:\Users\jose_\Documents\samsung_ic\datasets\innobrand_proyect\.venv\Scripts\python.exe infer/